# Limpieza de datos y preprocesado de textos

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata,string

PATH="/home/enriqueotero/datasets/nlp/corpusCriticasCine/"

def clean(x):
   x = unicodedata.normalize('NFKD', x).encode('ascii','ignore').lower()
   replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
   x = x.translate(replace_punctuation)
   x = re.sub('@%$&[\n/:!,;)()_?¿¡<>]', ' ', x)
   x = re.sub(' - ', ' ', x)
   x = re.sub(' +',' ', x).strip()
   return x

def parseFile(filename):
    
    try:
        data = open(filename,'r').read()
        soup = BeautifulSoup(data, "html.parser")  
        review_text = soup.body.get_text()
        letters_only = clean(review_text)
        #letters_only = re.sub("[^a-zA-Z]", " ", review_text) 

        result = letters_only

    except:
        result = ""
        
    return result

parseFile(PATH + str(999) + ".xml")

'bajo mi modesta opinion me resulta una pelicula casi documental a cerca de los acontecimientos sucedidos en la casa real inglesa a tenor de la muerte de lady di y muy especialmente visto desde el punto de vista de la riena isabel de inglaterra la pelicula es entretenida e interesante sobre todo por lo reciente de los acontecimientos pero cuando termino mi sensacion fue mas la de una intencion por parte del creador de la pelicula de reconciliar la opinion publica con la imagen de la monarquia francamente lo tomare como una vision romantica de lo que pudo pasar por la mente de la monarca tras la muerte de lady di y no como lo que realmente sucediera en cualquier caso me quedo con la excepcional interpretacion de helen mirren actriz que me era completamente desconocida hasta esta pelicula asi como el parecido fisico de la reina es asombroso queda muy lejos el reflejo del principe carlos en el actor que lo interpreta alex jennings de espaldas si que da el pego pero visto de cara creo que 

In [2]:
N_DOCS = 100
mydict = {"%i"%i:parseFile(PATH+str(i+1)+".xml") for i in range(N_DOCS)}
df = pd.DataFrame(data=mydict.items(), columns=['id', 'text'])
df.head()

,id,text
0,24,la ultima pelicula de sam mendes ganador del o...
1,25,el cine asiatico se ha convertido ya en lo que...
2,26,por fin he podido ver brokeback mountain la la...
3,27,
4,20,


In [3]:
df.to_csv(PATH+"clean_reviews.csv", index=False)

# Bag of Words

# TF-IDF

In [4]:
#corpus = ['This is the first document.','This is the second second document.','And the third one.','Is this the first document?']
corpus = df["text"].tolist()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
#vectorizer = TfidfVectorizer(analyzer=u'word', use_idf=True, max_df=0.3, lowercase=False)
vectorizer = TfidfVectorizer(analyzer=u'word', use_idf=True)
tfidf_matrix = vectorizer.fit_transform(corpus)

In [6]:
import operator
import numpy as np

def getDictOfKeywords(idx, top_n, tfidf_matrix, vectorizer):
    indices = np.argsort(tfidf_matrix[idx:idx+1].toarray().flatten())[::-1]
    features = vectorizer.get_feature_names()
    top_features = {features[i]:tfidf_matrix[idx:idx+1].toarray().flatten()[i] for i in indices[:top_n]}
    return top_features 

def firstkeywords(keywords, N):
    keywordlist = sorted(keywords.items(), key=operator.itemgetter(1), reverse=True)
    return map(lambda x: x[0],keywordlist[0:N])

# Extracción de palabras clave

In [8]:
N_KEYWORDS = 10
dictkeywords = getDictOfKeywords(3, N_KEYWORDS, tfidf_matrix, vectorizer)
keywords = firstkeywords(dictkeywords,N_KEYWORDS)
print keywords

[u'enamoro', u'en', u'encajan', u'enamora', u'encantan', u'enamorados', u'enamoramiento', u'zurra', u'encadenado', u'enamorado']
